# 🧠 Entrenamiento CNN en Google Colab
Este notebook permite entrenar un modelo CNN directamente en Google Colab usando imágenes almacenadas en tu Google Drive.

Incluye:
- Montaje de Google Drive
- Limpieza de imágenes corruptas
- Preprocesamiento con `ImageDataGenerator`
- Entrenamiento del modelo CNN
- Visualización de métricas
- Guardado del modelo entrenado en tu Drive

In [1]:
# 📂 Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# 📁 Ruta al dataset y al modelo
# Asegúrate de que tu estructura sea por carpetas: drive/MyDrive/mi_proyecto/dataset/train/happy, sad, etc.
dataset_path = '/content/drive/MyDrive/mi_proyecto/dataset/train'
modelo_path = '/content/drive/MyDrive/mi_proyecto/modelo_emociones.h5'

In [ ]:
# 🧹 Limpieza de imágenes corruptas
import os
from PIL import Image

def limpiar_imagenes_invalidas(ruta):
    for root, _, files in os.walk(ruta):
        for f in files:
            if f.lower().endswith((".png", ".jpg", ".jpeg")):
                path = os.path.join(root, f)
                try:
                    img = Image.open(path)
                    img.verify()
                except Exception as e:
                    print(f"🗑️ Eliminada: {path} — Motivo: {e}")
                    os.remove(path)

limpiar_imagenes_invalidas(dataset_path)

In [ ]:
# 🔄 Preprocesamiento del dataset
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = val_datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
# 🧠 Definición y compilación del modelo CNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam

num_clases = train_generator.num_classes

model = Sequential([
    Input(shape=(64, 64, 3)),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_clases, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# 🚀 Entrenamiento del modelo
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

In [ ]:
# 💾 Guardar el modelo entrenado en Google Drive
model.save(modelo_path)

In [ ]:
# 📊 Visualización de métricas
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Precisión')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Pérdida')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()
plt.show()